#### this script renames the 4th column of the promoter_motifs.bed to the Arabidopsis gene ID nomenclature (eg. AT4G3800)

In [1]:
import pandas as pd
#import argparse
from pybedtools import BedTool
import re
import numpy as np

In [ ]:
# parser = argparse.ArgumentParser(description='motif_rename')
# parser.add_argument('motifs_bed', type=str, help='Input location of *motifs.bed file')
# parser.add_argument('motifs_bed_renamed', type=str, help='Output location of promoter bed file')
# parser.add_argument('genome_gff', type=str, help='Input location of genome gff file')
# parser.add_argument('genome_db', type=str, help='Output location of genome gff database')

In [2]:
def rename_motif(motifs_bed):
    """function to add a TF name column in the motifs.bed file"""
    #read in motifs_bed_file
    motifs = pd.read_table(motifs_bed, sep='\t', header=None)
    cols = ['chr', 'start', 'stop', 'name_rep', 'score', 'strand']
    motifs.columns = cols
    motifs['TF'] = motifs.name_rep    
    capitalise = lambda x: x.upper()
    motifs.TF = motifs.TF.apply(capitalise)
    #replace characters upto and including the '.' in the TF column
    motifs.TF = motifs.TF.replace('^[^\.]*\.', '', regex=True)
    #replace characters after the '_'
    motifs.TF = motifs.TF.replace('_.*$', '', regex=True)
    return motifs

In [127]:
#depracated
# def create_geneID_table(uniprot_dat):
#     """This function uses the genomic .gff file to replace gene ALIAS names with gene ID's"""
#     uniprot_df = pd.read_table(uniprot_dat, sep='\t', header=None)
#     cols = ['code','category','ID']
#     uniprot_df.columns = cols
#     #drop duplicates of columns 'code' and 'category'
#     uniprot_no_dups = uniprot_df.sort_values('ID').drop_duplicates(subset=['code', 'category'], keep='last')
#     #pivot table based on the 'code' column
#     pivot = uniprot_no_dups.pivot(index='code', columns='category', values='ID')
#     #focus on columns of interest
#     geneID_table = pivot[['EnsemblGenome', 'Araport', 'Gene_OrderedLocusName', 'Gene_Name', 'Gene_Synonym']]    
#     return geneID_table

In [149]:
#depracated
# def map_ID(renamed_motifs_df, geneIDtable, output_bed):
#     """This function uses the geneIDtable to replace gene ALIAS names in a dataframe with gene ID's"""
#     #renamed_motifs_df.TF
#     #mapped_IDs = renamed_motifs_df.TF
#     #renamed_motifs_df['gene_id'] = renamed_motifs_df.apply(lambda x: (x.TF in geneIDtable.Gene_Name), axis='columns')
#     #renamed_motifs_df['gene_id'] = renamed_motifs_df.TF in geneIDtable.Gene_Name
#     #change columns to uppercase
#     motifs = renamed_motifs_df
#     motifs.TF = renamed_motifs_df.TF.apply(lambda x: str(x).upper())
# #     geneIDtable.Gene_Name = geneIDtable.Gene_Name.apply(lambda x: str(x).upper())
# #     geneIDtable.Gene_OrderedLocusName = geneIDtable.Gene_OrderedLocusName.apply(lambda x: str(x).upper())
# #     geneIDtable.Araport = geneIDtable.Araport.apply(lambda x: str(x).upper())
# #     geneIDtable.EnsemblGenome = geneIDtable.EnsemblGenome.apply(lambda x: str(x).upper())   
# #     geneIDtable.Gene_Synonym = geneIDtable.Gene_Synonym.apply(lambda x: str(x).upper())
#     #merged1 = pd.merge(motifs, geneIDtable, how='left', left_on = 'TF', right_on = 'Gene_Name')
# #     motifs['gene_id'] = motifs[motifs.TF == geneIDtable.Gene_Name]
# #     #motifs['gene_id'] = motifs.apply(lambda x: (str(x.TF) in str(geneIDtable.Gene_Name)), axis='columns')
# #     for index, row in renamed_motifs_df.iterrows():
# #         if row.TF in (geneIDtable.Gene_Name.upper()):
# # #             print(row.TF)    
    
# #     #elif renamed_motifs_df.TF.isin(geneIDtable.Gene_Synonym):
            
         
#     #mapped_IDs = renamed_motifs_df.apply(lambda x: x[''])
#     #return motifs[motifs['gene_id']==True]
#     return motifs 

In [13]:
def json2table(json_file):
    """function to read in json file and convert it to a table"""
    mmt_df_full = pd.read_json(json_file, orient='record')
    return mmt_df_full

In [3]:
def map_ID2(motifs, geneIDtable, output_bed):
   
    """function to rename the TF column values in a motifs.bed file to the Arabidopsis gene ID nomenclature"""
    #print(motifs.shape)
    #remove '_m' from end of name_rep value in motifs
    motifs.name_rep = motifs.name_rep.str.replace('_m1', '')
    merged = pd.merge(motifs,geneIDtable, on='name_rep')
    #print(merged.shape)
    #make bed file
    sorted_motifs = merged.sort_values(['chr','start'])
    bed = BedTool.from_dataframe(sorted_motifs).saveas(output_bed)


In [ ]:
#downloaded table from http://neomorph.salk.edu/dev/pages/shhuang/dap_web/pages/dap-grid-data3.php?_=1573587374712

#### I downloaded the mapping table from 'http://neomorph.salk.edu/dev/pages/shhuang/dap_web/pages/browse_table_aj.php'. There was no download for this so to do this, I inspected the element in chrome and found the link 'http://neomorph.salk.edu/dev/pages/shhuang/dap_web/pages/dap-grid-data3.php?_=1573587374712'. Then I downloaded the .html but named it dap-grid-data3.json. To make it into .json format, I removed the first line '<html><head>'. On the second line I removed up until the '[' after data (removed '<meta http-equiv="content-type" content="text/html; charset=UTF-8"></head><body>{"data":'. I removed '}</body></html>' from the end making sure the start and end were encapsulated by square brackets [].

### point to file locations:

In [10]:
motif_mapping_table = '../../data/FIMO/motif_data/dap-grid-data3.json'
motifs_bed = '../../data/FIMO/promoters_renamedChr_motifs.bed'
motifs_bed_mapped = '../../data/FIMO/promoters_renamedChr_motifs_mapped.bed'

In [203]:
#mmt_df_full = pd.read_json(motif_mapping_table, orient='record')
#mmt_df = mmt_df[['name_rep', 'at_id']]
mmt_df.to_csv('../../data/FIMO/motif_data/motif_map_IDs.txt', index=False, sep='\t')


In [202]:
mmt_df_full = pd.read_json(motif_mapping_table, orient='record')
mmt_df_full.to_csv('../../data/FIMO/motif_data/motif_map.txt', index=False, sep='\t')

### run functions:

In [223]:
motifs_renamed = rename_motif(motifs_bed)

In [14]:
mmt_df_full = json2table(motif_mapping_table)

In [15]:
mmt_df_motif_IDs = mmt_df_full[['name_rep', 'at_id']]

In [16]:
mmt_df_motif_IDs

,name_rep,at_id
0,ABI3VP1_tnt.AT5G18090_col_a,AT5G18090
1,ABI3VP1_tnt.AT5G25475_col_a,AT5G25475
2,ABI3VP1_tnt.AT5G60130_col_a,AT5G60130
3,ABI3VP1_tnt.AT5G60130_colamp_a,AT5G60130
4,ABI3VP1_tnt.FUS3_col_a,AT3G26790
...,...,...
867,bZIP_tnt.bZIP68_col_a,AT1G32150
868,bZIP_tnt.bZIP69_col_a,AT1G06070
869,bZIP_tnt.bZIP69_colamp_a,AT1G06070
870,mTERF_tnt.AT5G23930_col_a,AT5G23930


In [232]:
map_ID2(motifs_renamed, mmt_df_motif_IDs, motifs_bed_mapped)

#### write out the motif_mapping_table to file (motif_map.txt) and also the one with filtered columns to motif_map_IDs.txt

In [ ]:
mmt_df_motif_IDs.to_csv('../../data/FIMO/motif_data/motif_map_IDs.txt', index=False, sep='\t')

In [ ]:
mmt_df_full.to_csv('../../data/FIMO/motif_data/motif_map.txt', index=False, sep='\t')

In [131]:
#geneID_table = create_geneID_table(uniprot_database) 
#not using uniprot database anymore

In [151]:
print(geneID_table[['Gene_Name', 'Gene_Synonym']])

category    Gene_Name Gene_Synonym
code                              
A0A023T4L8        LFY          NAN
A0A060D9F2    FRIGIDA          NAN
A0A068FHR7        NAN          NAN
A0A068FHS2        NAN          NAN
A0A068FIL1        NAN          NAN
...               ...          ...
X5JAD2         SEC10B          NAN
X5JAK7      AT5G12350          NAN
X5JAK8         ANNAT8    AT5G12380
X5JAX7      AT5G12390    FISSION1B
X5JB51         SEC10B          NAN

[94598 rows x 2 columns]


In [5]:
motifs_renamed = rename_motif('../../data/FIMO/responsivepromoters_motifs.bed')

In [6]:
motifs_renamed

,chr,start,stop,name_rep,score,strand,TF
0,1,33629,33647,ABI3VP1_tnt.VRN1_colamp_a_m1,7.86301,-,VRN1
1,1,33961,33979,ABI3VP1_tnt.VRN1_colamp_a_m1,10.71230,+,VRN1
2,1,33962,33982,REM_tnt.REM19_colamp_a_m1,11.16180,-,REM19
3,1,33964,33982,ABI3VP1_tnt.VRN1_colamp_a_m1,10.35620,+,VRN1
4,1,34064,34084,ND_tnt.FRS9_col_a_m1,7.51562,+,FRS9
...,...,...,...,...,...,...,...
16242,5,26957500,26957518,C2H2_tnt.TF3A_col_a_m1,11.18640,-,TF3A
16243,5,26957503,26957521,C2H2_tnt.TF3A_col_a_m1,12.82200,-,TF3A
16244,5,26957535,26957563,BBRBPC_tnt.BPC5_colamp_a_m1,3.36047,+,BPC5
16245,5,26957541,26957564,BBRBPC_tnt.BPC1_col_a_m1,1.12500,+,BPC1


In [8]:
map_ID2(motifs_renamed, '../../data/FIMO/motif_data/motif_map_IDs.txt', '../../data/FIMO/responsivepromoters_motifs_mapped.bed')

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed